In [9]:
import ollama
import json
import pandas as pd

## Define Schema

In [84]:
schema = """
Dataset fields:

MODES:
- bus, national-rail, tube
- bus, tube
- tube
- tube, bus

ACCESSIBILITY:
- Not Accessible
- Partially Accessible
- Partially Accessible - Interchange Only
- Fully Accessible

NIGHT_TUBE:
- Yes
- No
"""

## Function to convert text -> WHERE clause

In [85]:
def convert_to_where(user_input):
    prompt = f"""
You are generating a pandas DataFrame.query() expression.

Return ONLY the expression.
Do NOT include:
- DataFrame.query(
- df.query(
- WHERE
- explanations
- markdown
- surrounding quotes

Rules:
- Use == for equality
- Use and / or in lowercase
- Include spaces around AND / OR
- Field names are case-sensitive and must match exactly
- Default: include all tube stations. Only filter for bus or national rail if the user specifically asks. 

Valid fields:
- MODES
- ACCESSIBILITY
- NIGHT_TUBE

{schema}

User request:
{user_input}

WHERE clause:
"""
    
    response = ollama.chat(
        model = 'llama3',
        messages=[
            {"role": "user", "content": prompt}
        ]
    )

    return response['message']['content'].strip()

## Interactive loop

In [42]:
if __name__ == "__main__":
    while True:
        user_text = input("\nEnter query (or 'quit'): ")
        if user_text.lower() == "quit":
            break

        where_clause = convert_to_where(user_text)
        print("\nGenerated WHERE clause:")
        print(where_clause)


Generated WHERE clause:
Accessibility == 'Fully Accessible'


## Test Filtering with dataset

In [91]:
df = pd.read_csv("data/Underground_Stations/Underground_Stations.csv")

In [94]:
user_input = "show me all fully accessible stations with night tube or those stations that has bus stops"

where_clause = convert_to_where(user_input)

print("Generated expression: ", where_clause)

filtered = df.query(where_clause)

filtered.head()


Generated expression:  ACCESSIBILITY == 'Fully Accessible' and (NIGHT_TUBE == 'Yes' or MODES.str.contains('bus'))


,X,Y,OBJECTID,NAME,LINES,ATCOCODE,MODES,ACCESSIBILITY,NIGHT_TUBE,NETWORK,DATASET_LAST_UPDATED,FULL_NAME
11,540949.9810,191740.0184,122,Woodford,Central,940GZZLUWOF,"tube, bus",Fully Accessible,Yes,London Underground,2021/11/29 00:00:00+00,Woodford station
17,542463.3563,184275.4352,128,East Ham,"District, Hammersmith & City",940GZZLUEHM,"bus, tube",Fully Accessible,No,London Underground,2021/11/29 00:00:00+00,East Ham station
22,545845.6808,184254.3449,133,Upney,District,940GZZLUUPY,"tube, bus",Fully Accessible,No,London Underground,2021/11/29 00:00:00+00,Upney station
27,525255.0926,187480.2837,138,Golders Green,Northern,940GZZLUGGN,"tube, bus",Fully Accessible,Yes,London Underground,2021/11/29 00:00:00+00,Golders Green station
28,525278.0259,190668.3440,139,Finchley Central,Northern,940GZZLUFYC,"tube, bus",Fully Accessible,Yes,London Underground,2021/11/29 00:00:00+00,Finchley Central station
